In [ ]:
# -*- coding: utf-8 -*-
# install libraries
%pip install --upgrade tiktoken
%pip install --upgrade openai
%pip install --upgrade python-dotenv

In [23]:
# import libraries
import os
import time
import pprint
import tiktoken
import concurrent.futures
from dotenv import load_dotenv
from openai import AzureOpenAI

# Funções que serão utilizadas durante os exemplos

In [24]:
# Nome das regiões Azure OpenAI
EASTUS = "eastus"
CANADAEAST = "canadaeast"

# Nome dos modelos Azure OpenAI criados na subscrição 
GPT_35_TURBO = "gpt-35-turbo"
GPT_35_TURBO_16K = "gpt-35-turbo-16k"
GPT_35_TURBO_INSTRUCT = "gpt-35-turbo-instruct"

GPT_4 = "gpt-4"
GPT_4_32K = "gpt-4-32k"
GPT_4_TURBO = "gpt-4-turbo"
GPT_4o = "gpt-4o"

DALL_E_3 = "dall-e-3"
TEXT_EMBEDDING_ADA_002 = "text-embedding-ada-002"
TEXT_EMBEDDING_ADA_3_SMALL = "text-embedding-3-small"
TEXT_EMBEDDING_ADA_3_LARGE = "text-embedding-3-large"

load_dotenv() # carregar variáveis de ambiente

deployments_in_regions = {"CANADAEAST": [GPT_35_TURBO, GPT_4, GPT_4_32K, GPT_4_TURBO, TEXT_EMBEDDING_ADA_002, TEXT_EMBEDDING_ADA_3_SMALL],
                          "EASTUS": [GPT_35_TURBO_16K, GPT_35_TURBO_INSTRUCT, GPT_4o, DALL_E_3, TEXT_EMBEDDING_ADA_002, TEXT_EMBEDDING_ADA_3_LARGE]}


# carregar tokenizador para os modelos de linguagem
encoding = tiktoken.get_encoding("cl100k_base")

# Exemplos de system messages
papel = {"bot": "Você é um assistente virtual, capaz de responder perguntas, fornecer informações e que sempre responde em formato JSON.",
        "escritor": "Você é um especialista em escrita e redação, capaz de escrever, compreender e analisar textos em português, especialmente textos opinativos e argumentativos.",
        "legislador": "Você é um especialista em legislação, capaz de escrever, interpretar e analisar leis e normas jurídicas em português.",
        "comediante": "Você é um comediante, capaz de criar piadas, contar histórias engraçadas e fazer humor em português.",
        "desevolvedor": "Você é um desenvolvedor de software, capaz de criar, testar e manter programas de computador em qualquer linguagem, mas em especial em Python, HTML e Javascript",
        "hacker": "Você é um hacker, capaz de invadir sistemas de computadores, roubar informações e burlar sistemas de segurança.",
        "professor": "Você é um professor, capaz de ensinar e explicar conceitos, teorias e práticas em qualquer disciplina. Com explicações claras e didáticas, com exemplos práticos e exercícios para fixação dos conceitos.",
        "instrutor": "Você é um instrutor, capaz de ensinar e treinar pessoas em qualquer área de conhecimento. Suas explicações são diretas, sucintas e sem rodeios.",
        "medico": "Você é um médico, capaz de diagnosticar, tratar e prevenir doenças em seres humanos e animais.",
        "advogado": "Você é um advogado, capaz de representar clientes em processos judiciais, elaborar contratos e prestar consultoria jurídica.",
        "padre": "Você é um padre, capaz de ministrar sacramentos, celebrar missas e aconselhar fiéis em questões espirituais.",
        "rabino": "Você é um rabino, capaz de ministrar cerimônias religiosas, ensinar a Torá e aconselhar fiéis em questões espirituais.",
        "pastor": "Você é um pastor, capaz de ministrar cultos, pregar a Bíblia e aconselhar fiéis em questões espirituais.",}



# inicialização do cliente Azure OpenAI Canada East
client_canadaeast = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT_CANADAEAST"), 
    api_key=os.getenv("AZURE_OPENAI_KEY_CANADAEAST"),  
    api_version="2024-02-15-preview"
)

# inicialização do cliente Azure OpenAI East US
client_eastus = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT_EASTUS"), 
    api_key=os.getenv("AZURE_OPENAI_KEY_EASTUS"),  
    api_version="2024-02-15-preview"
)

# funcao para retornar o cliente de acordo com a região
def get_client(region):
    if region == "canadaeast":
        return client_canadaeast
    elif region == "eastus":
        return client_eastus
    else:
        return None

# função para chamar o modelo de linguagem
def call_llm(deployment_name, system_message, question, region="canadaeast"):
    #verificar se o modelo está disponível na região
    if deployment_name not in deployments_in_regions[region.upper()]:
        return {"0.error": f"Model {deployment_name} not available in region {region}", 
                "1.This regions has the following models available": deployments_in_regions[region.upper()], 
                f"2.The model {deployment_name} is available in the following regions": [region for region, models in deployments_in_regions.items() if deployment_name in models]}
    
    client = get_client(region)
    start_time = time.perf_counter()
    response = client.chat.completions.create(
        model = deployment_name,
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": question},
        ]
    )
    elapsed_time = time.perf_counter() - start_time # usado para calcular o tempo de execução
    resp = response.choices[0].message.content
    tokens_completion = num_tokens_from_string(resp, deployment_name) # conta os tokens da resposta
    # conta os tokens da mensagem do sistema e da pergunta (prompt)
    tokens_prompt = num_tokens_from_string(system_message, deployment_name) + num_tokens_from_string(question, deployment_name)
    # formatar a resposta da função
    return {"0.model": deployment_name, 
            "1.elapsed_time": elapsed_time, 
            "2.response": resp, 
            "3.num_tokens_completion": tokens_completion, 
            "4.num_tokens_prompt": tokens_prompt,
            "5.system_message": system_message,
            "6.question": question}

# funcao para contar tokens
def num_tokens_from_string(texto, model):
    encoding = tiktoken.encoding_for_model(model)
    num_tokens = len(encoding.encode(texto))
    return num_tokens

# função para gerar embeddings
def generate_embeddings(text, model, region="canadaeast"):
    client = get_client(region)
    start_time = time.perf_counter()
    embeddings = client.embeddings.create(input = [text], model=model).data[0].embedding
    elapsed_time = time.perf_counter() - start_time
    return embeddings, elapsed_time

# função para executar modelos simultaneamente
# models: lista de modelos
# system_message: mensagem do sistema
# question: pergunta
# retorna uma lista com os resultados
def execute_simultaneously(models, system_message, question, debug=False, region="canadaeast"):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = [executor.submit(call_llm, model, system_message, question, region) for model in models]
        results = []
        for future in concurrent.futures.as_completed(futures):
            if debug:
                print(future.result())
            results.append(future.result())
    return results

def print_result(result, debug=False):
    print("Elapsed time:", result["1.elapsed_time"])
    print("Response:", result["2.response"])
    print("Num tokens completion:", result["3.num_tokens_completion"])
    print("Num tokens prompt:", result["4.num_tokens_prompt"])
    if debug:
        print("System message:", result["5.system_message"])
        print("Question:", result["6.question"])
        print("Model:", result["0.model"])
        print("===========================================================\n")  

#funcao para imprimir resultados
def print_results(system_message, question, results, debug=False):
    if debug:
        print("System message:", system_message)
        print("Question:", question)

    for result in results:
        print("Model:", result["0.model"], "Elapsed time:", result["1.elapsed_time"])
        print("Response:", result["2.response"])
        print("Num tokens completion:", result["3.num_tokens_completion"], 
              ", Num tokens prompt:", result["4.num_tokens_prompt"])
        print("===========================================================\n")

In [26]:
def sample_agenda():
    agenda = '''12/03/2022 12:00:00 Ocupado 12/03/2022 13:00:00 Ocupado 12/03/2022 14:00:00 Ocupado 12/03/2022 15:00:00 Livre 12/03/2022 16:00:00 Ocupado 12/03/2022 17:00:00 Livre
        12/03/2022 18:00:00 Ocupado 12/03/2022 19:00:00 Livre 12/03/2022 20:00:00 Ocupado'''
    return call_llm(GPT_35_TURBO, papel["bot"], 
                    f"Com base nessa agenda, qual o próximo horário livre? {agenda}", "canadaeast")

resposta = sample_agenda()
print_result(resposta)

Elapsed time: 1.0153741289996105
Response: {
  "próximo_horário_livre": "12/03/2022 15:00:00"
}
Num tokens completion: 26
Num tokens prompt: 182


# Exemplo de diferentes tons na resposta

In [27]:
def sample_system_message_intonation(question):
    system_message = {"despojado":"Você é um assitente que sempre responde em tom despojado, de forma didática e em porguguês, apesar de eventualmente citar palavras em inglês.",
                      "formal":"Você é um assistente que sempre responde em tom formal, rebuscado, sem citar palavras em inglês.",
                      "informal":"Você é um assistente que sempre responde em tom informal, de forma didática e em português, com gírias e expressões coloquiais."}
    

    for item in system_message:
        print(f"Resposta com tom {item.upper()}:")
        r = call_llm(GPT_35_TURBO, system_message[item], question, "canadaeast")
        pprint.pprint(r["2.response"])

sample_system_message_intonation("O que é inteligência artificial?")

Resposta com tom DESPOJADO:
('Inteligência artificial, ou IA, é quando as máquinas são programadas para '
 'simular a capacidade humana de aprender, raciocinar, tomar decisões e '
 'resolver problemas. Isso é feito através de algoritmos e modelos matemáticos '
 'que permitem que as máquinas processem grandes quantidades de dados e '
 'aprendam com eles, de forma a tomar decisões ou realizar tarefas de forma '
 'autônoma. Em resumo, é como se a máquina tivesse um cérebro próprio, capaz '
 'de aprender e tomar decisões como um ser humano. Dá para dizer que é uma '
 'espécie de "mindset" tecnológico, saca?')
Resposta com tom FORMAL:
('A inteligência artificial é um ramo da ciência da computação que se dedica '
 'ao desenvolvimento de sistemas capazes de realizar tarefas que, comumente, '
 'exigiriam inteligência humana. Estes sistemas são projetados para aprender, '
 'raciocinar, resolver problemas e tomar decisões de forma autônoma, sempre '
 'com o objetivo de simular a capacidade cogni

# Exemplo de diferença entre modelos

In [28]:
system_message = papel["instrutor"]
question = "Conte-me um paradoxo de maneira estrudurada e sucinta com o mínimo de detalhes possível."
models = [GPT_35_TURBO, GPT_4, GPT_4_32K, GPT_4_TURBO]
results = execute_simultaneously(models, system_message, question, True)
print_results(system_message, question, results)

{'0.model': 'gpt-35-turbo', '1.elapsed_time': 1.3253236370001105, '2.response': 'Um paradoxo é uma declaração que parece contraditória, mas pode ser verdadeira. Um exemplo famoso é o Paradoxo do Mentiroso, que é a afirmação "Esta frase é falsa". Se for verdadeira, então é falsa, mas se for falsa, então é verdadeira.', '3.num_tokens_completion': 68, '4.num_tokens_prompt': 63, '5.system_message': 'Você é um instrutor, capaz de ensinar e treinar pessoas em qualquer área de conhecimento. Suas explicações são diretas, sucintas e sem rodeios.', '6.question': 'Conte-me um paradoxo de maneira estrudurada e sucinta com o mínimo de detalhes possível.'}
{'0.model': 'gpt-4-32k', '1.elapsed_time': 12.598692859000039, '2.response': 'Paradoxo do avô: Imagine um cenário em que viagens no tempo são possíveis. Você volta no tempo e mata seu avô antes que ele conheça sua avó. Como resultado, um dos seus pais (e, consequentemente, você) não deveria existir. Porém, se você não existir, você não poderia vol

In [36]:
papel.keys()

dict_keys(['bot', 'escritor', 'legislador', 'comediante', 'desevolvedor', 'hacker', 'professor', 'instrutor', 'medico', 'advogado', 'padre', 'rabino', 'pastor'])

# E aí, qual o seu prompt?

In [ ]:
# faça uma pergunta ao modelo, seja criativo!
system_message = papel["desevolvedor"]


prompt = f''' '''

response = call_llm(GPT_4_32K, system_message, prompt)
response["2.response"]

In [ ]:
print_result(response)